# Completing information about medicines in pharmacy

---

## Imports

#### Standard library imports

#### Third party imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

#### Local application imports

---

## Data

### Data to complete

#### Reading data

In [4]:
df_obj = pd.read_csv("../data/cat_plat_pc.csv")

In [5]:
df_obj

,clave,medicamento
0,6.5024E+11,CICATRICURE 1 GEL DESVANECEDOR 30 G
1,6.5024E+11,SILKAMEDIC GEL 30G/ TERBINAFINA 1%
2,6.5024E+11,M FORCE & MULTI-O 3 PIEZAS CARTERA
3,6.5024E+11,SHAMPOO MEDICASP BOTE 130 ML
4,6.5024E+11,"bio electro 24 tab250/250/65 mg /paracetamol ,"
...,...,...
2212,7.50135E+12,MOMETASONA 0.5% 18 ML
2213,7.5013E+12,DEXABION C/3 JERINGAS DOBLE CAMARA 3 ML
2214,7.50131E+12,MICCIL 1 MG C/20
2215,7.50135E+12,ROSUVASTATINA 20 MG C/15 TAB


#### Cleaning data

In [32]:
df_obj["medicamento"] = df_obj["medicamento"].str.upper()

### Reference data

#### Reading data

In [38]:
df_ref = pd.read_csv("../data/cat_fansa.csv")

In [39]:
df_ref

,cod_barras,descripcion,tipo_descuento,laboratorio
0,7.36085E+11,3 A OFTENO 1MG 5ML - GOT 5ML,BASE,SOPHIA
1,7.59684E+11,360 ANTISEPTICO JALOMA - SOL 150ML,NETO,LABORATORIOS JALOMA
2,75313036,360 ANTISEPTICO JALOMA - SOL 30ML,NETO,LABORATORIOS JALOMA
3,4.04696E+12,4002801 ASKINA BARRIER FILM SPRA - 28ML,NETO,B BRAUN MEDICAL
4,4.04696E+12,400590 PRONTOSAN WOUND - GEL 30ML,NETO,B BRAUN MEDICAL
...,...,...,...,...
8105,7.50114E+12,ZYVOXAM 600MG (A) - TAB 10,NETO,UPJOHN PHARMA MEXICO S DE RL DE CV
8106,7.50109E+12,ZYXEM 5MG - TAB 30,NETO,CHINOIN
8107,7.50109E+12,ZYXEM 5MG - TAB 10,NETO,CHINOIN
8108,7.50109E+12,ZYXEM 5MG/ML - SOL 20ML,NETO,CHINOIN


#### Cleaning data

In [40]:
col_splt = df_ref["descripcion"].str.split(" - ", 1, expand=True)
df_ref["med_ref"] = col_splt[0]
df_ref["pres_red"] = col_splt[1]
df_ref.drop(["descripcion"], axis=1, inplace = True)

In [41]:
df_ref

,cod_barras,tipo_descuento,laboratorio,med_ref,pres_red
0,7.36085E+11,BASE,SOPHIA,3 A OFTENO 1MG 5ML,GOT 5ML
1,7.59684E+11,NETO,LABORATORIOS JALOMA,360 ANTISEPTICO JALOMA,SOL 150ML
2,75313036,NETO,LABORATORIOS JALOMA,360 ANTISEPTICO JALOMA,SOL 30ML
3,4.04696E+12,NETO,B BRAUN MEDICAL,4002801 ASKINA BARRIER FILM SPRA,28ML
4,4.04696E+12,NETO,B BRAUN MEDICAL,400590 PRONTOSAN WOUND,GEL 30ML
...,...,...,...,...,...
8105,7.50114E+12,NETO,UPJOHN PHARMA MEXICO S DE RL DE CV,ZYVOXAM 600MG (A),TAB 10
8106,7.50109E+12,NETO,CHINOIN,ZYXEM 5MG,TAB 30
8107,7.50109E+12,NETO,CHINOIN,ZYXEM 5MG,TAB 10
8108,7.50109E+12,NETO,CHINOIN,ZYXEM 5MG/ML,SOL 20ML


---

## Matching data

In [42]:
df_obj

,clave,medicamento
0,6.5024E+11,CICATRICURE 1 GEL DESVANECEDOR 30 G
1,6.5024E+11,SILKAMEDIC GEL 30G/ TERBINAFINA 1%
2,6.5024E+11,M FORCE & MULTI-O 3 PIEZAS CARTERA
3,6.5024E+11,SHAMPOO MEDICASP BOTE 130 ML
4,6.5024E+11,"BIO ELECTRO 24 TAB250/250/65 MG /PARACETAMOL ,"
...,...,...
2212,7.50135E+12,MOMETASONA 0.5% 18 ML
2213,7.5013E+12,DEXABION C/3 JERINGAS DOBLE CAMARA 3 ML
2214,7.50131E+12,MICCIL 1 MG C/20
2215,7.50135E+12,ROSUVASTATINA 20 MG C/15 TAB


In [43]:
df_ref

,cod_barras,tipo_descuento,laboratorio,med_ref,pres_red
0,7.36085E+11,BASE,SOPHIA,3 A OFTENO 1MG 5ML,GOT 5ML
1,7.59684E+11,NETO,LABORATORIOS JALOMA,360 ANTISEPTICO JALOMA,SOL 150ML
2,75313036,NETO,LABORATORIOS JALOMA,360 ANTISEPTICO JALOMA,SOL 30ML
3,4.04696E+12,NETO,B BRAUN MEDICAL,4002801 ASKINA BARRIER FILM SPRA,28ML
4,4.04696E+12,NETO,B BRAUN MEDICAL,400590 PRONTOSAN WOUND,GEL 30ML
...,...,...,...,...,...
8105,7.50114E+12,NETO,UPJOHN PHARMA MEXICO S DE RL DE CV,ZYVOXAM 600MG (A),TAB 10
8106,7.50109E+12,NETO,CHINOIN,ZYXEM 5MG,TAB 30
8107,7.50109E+12,NETO,CHINOIN,ZYXEM 5MG,TAB 10
8108,7.50109E+12,NETO,CHINOIN,ZYXEM 5MG/ML,SOL 20ML


---
---